In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.utils import resample
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
np.random.seed(12)

In [2]:
# Participants with above-median genetic risks for depression
data = pd.read_csv('./high_genetic/data_predict_high_genetic.csv')
print(data['emotion_group'].value_counts())

print(data.columns)

0    110
1    105
Name: emotion_group, dtype: int64
Index(['an1_time14', 'an2_time14', 'an3_time14', 'ap1_time14', 'ap2_time14',
       's1', 's2', 's3', 's4', 's5', 's6', 's7', 'hand', 'puberty', 'bmi14',
       'ses', 'ctqabuse', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'mdd_mean', 'semotion_14', 'ctq_cutoff', 'emotion_group'],
      dtype='object')


In [3]:
def data_process(data, train_index, test_index):
    train_data = data.iloc[train_index, :]
    test_data = data.iloc[test_index, :]
    all_colname = train_data.columns
    # select variables
    select_cols = list(all_colname[:16]) # for all factors + basic covariates

    # childhood abuse
    select_cols.append(all_colname[16])

    # emotional symptoms at age 14
    select_cols.append(all_colname[26])

    # emotional disorders
    select_cols.append(all_colname[28])

    train_data = train_data[select_cols]
    test_data = test_data[select_cols]
    
    # Normalization
    attr_list = select_cols[:5] + select_cols[13:-1] 

    # print(attr_list)
    scaler_list = dict()
    for i in attr_list:
        tmp = train_data[i]
        scaler = StandardScaler()
        train_data[i] = scaler.fit_transform(tmp.to_numpy().reshape(-1, 1))
        scaler_list[i] = scaler
    for i in attr_list:
        tmp_test = test_data[i]
        scaler = scaler_list[i]
        test_data[i] = scaler.transform(tmp_test.to_numpy().reshape(-1, 1))

    train_x, train_y = train_data.drop('emotion_group', axis=1), train_data['emotion_group']
    test_x, test_y = test_data.drop('emotion_group', axis=1), test_data['emotion_group']

    train_x['interaction'] = [i * j for i, j in zip(train_x['ap1_time14'], train_x['ctqabuse'])]
    test_x['interaction'] = [i * j for i, j in zip(test_x['ap1_time14'], test_x['ctqabuse'])]

    return train_x, train_y, test_x, test_y

In [4]:
def prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=None, threshold=0.5):
    if drop_features is not None:
         train_x, test_x = train_x.drop(drop_features,axis=1), test_x.drop(drop_features,axis=1)  
    
    model = model_pre.fit(train_x, train_y)
    y_pred = (model.predict_proba(test_x)[:, 1] > threshold).astype(bool)
    y_pred_proba = model.predict_proba(test_x)[:, 1]
    auc, recall = round(roc_auc_score(test_y, y_pred_proba), 4), recall_score(test_y, y_pred, average=None)
    
    return auc, recall[0], recall[1] 

**-------------- Prediction model for emotional disorders --------------------**

In [7]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Repeat a 5-fold cross-validation 10 times
total_auc = np.zeros((10, 5, 10))
total_spe = np.zeros((10, 5, 10))
total_sen = np.zeros((10, 5, 10))

for step in range(10):
    print(f"########################## {step + 1} ##########################")
    skf = StratifiedKFold(n_splits=5, shuffle=True)

    model_pre= LogisticRegression(max_iter=10000, class_weight='balanced')
#   model_pre = svm.SVC(kernel="linear", probability=True, class_weight='balanced')

    auc_result = np.zeros((5, 10))
    spe_result = np.zeros((5, 10))
    sen_result = np.zeros((5, 10))
    
    for num, (train_index, test_index) in enumerate(skf.split(data.drop('emotion_group', axis=1),  data['emotion_group'])):
        
        train_x, train_y, test_x, test_y = data_process(data, train_index, test_index)
        
        ###### all factors + interaction + abuse ########
        auc_result[num, 0], spe_result[num, 0], sen_result[num, 0] = \
        prediction(model_pre, train_x, train_y, test_x, test_y)
        
        ##### 3 factors + interaction + abuse ########
        drop_features = ['an2_time14', 'an3_time14']
        auc_result[num, 1], spe_result[num, 1], sen_result[num, 1] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### 1 factors + interaction + absue ########
        drop_features = ['an1_time14', 'an2_time14', 'an3_time14', 'ap2_time14']
        auc_result[num, 2], spe_result[num, 2], sen_result[num, 2] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### all factors + absue ########
        drop_features = ['interaction']
        auc_result[num, 3], spe_result[num, 3], sen_result[num, 3] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### 3 factors + abuse ########"
        drop_features = ['interaction', 'an2_time14', 'an3_time14']
        auc_result[num, 4], spe_result[num, 4], sen_result[num, 4] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### 1 factors + abuse ########
        drop_features = ['interaction', 'an1_time14', 'an2_time14', 'an3_time14', 'ap2_time14']
        auc_result[num, 5], spe_result[num, 5], sen_result[num, 5] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### all factors ########
        drop_features = ['interaction', 'ctqabuse']
        auc_result[num, 6], spe_result[num, 6], sen_result[num, 6] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### 3 factors ########
        drop_features = ['interaction', 'ctqabuse', 'an2_time14', 'an3_time14']
        auc_result[num, 7], spe_result[num, 7], sen_result[num, 7] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### 1 factors ########
        drop_features = ['interaction', 'ctqabuse', 'an1_time14', 'an2_time14', 'an3_time14', 'ap2_time14']
        auc_result[num, 8], spe_result[num, 8], sen_result[num, 8] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
        ##### without ########
        drop_features = ['an1_time14', 'an2_time14', 'an3_time14', 'ap1_time14', 'ap2_time14','ctqabuse', 'interaction']
        auc_result[num, 9], spe_result[num, 9], sen_result[num, 9] = \
        prediction(model_pre, train_x, train_y, test_x, test_y, drop_features=drop_features)
        
    
    total_auc[step] = auc_result
    total_spe[step] = spe_result
    total_sen[step] = sen_result

total_auc = total_auc.reshape(50, 10)
total_spe = total_spe.reshape(50, 10)
total_sen = total_sen.reshape(50, 10)

auc_pd = pd.DataFrame(total_auc, columns=['all_inter_abuse', 'three_inter_abuse', 'one_inter_abuse',
                                         'all_abuse', 'three_abuse', 'one_abuse', 'all', 'three', 'one', 
                                         'without'])

spe_pd = pd.DataFrame(total_spe, columns=['all_inter_abuse', 'three_inter_abuse', 'one_inter_abuse',
                                         'all_abuse', 'three_abuse', 'one_abuse', 'all', 'three', 'one', 
                                         'without'])

sen_pd = pd.DataFrame(total_sen, columns=['all_inter_abuse', 'three_inter_abuse', 'one_inter_abuse',
                                         'all_abuse', 'three_abuse', 'one_abuse', 'all', 'three', 'one', 
                                         'without'])

print("############ AUC ###############")
for column in auc_pd.columns:
        print(f'{column} | {auc_pd.mean()[column]} - {auc_pd.std()[column]}')
        
print("############ Specificity ###############")
for column in spe_pd.columns:
        print(f'{column} | {spe_pd.mean()[column]} - {spe_pd.std()[column]}')

print("############ Sensitivity ###############")
for column in sen_pd.columns:
        print(f'{column} | {sen_pd.mean()[column]} - {sen_pd.std()[column]}')




########################## 1 ##########################
########################## 2 ##########################
########################## 3 ##########################
########################## 4 ##########################
########################## 5 ##########################
########################## 6 ##########################
########################## 7 ##########################
########################## 8 ##########################
########################## 9 ##########################
########################## 10 ##########################
############ AUC ###############
all_inter_abuse | 0.7394379999999999 - 0.08136871958271565
three_inter_abuse | 0.747402 - 0.07544314385749072
one_inter_abuse | 0.7592619999999997 - 0.0750869669660672
all_abuse | 0.7329859999999999 - 0.08145763644185777
three_abuse | 0.7381840000000001 - 0.07790101086556932
one_abuse | 0.749654 - 0.07684080338795349
all | 0.7287939999999997 - 0.08786432818469017
three | 0.7342019999999999 - 0.085902050